In [1]:
import pandas as pd
df=pd.read_csv('dataset.csv ')
df.head()

,mark,model,description,fuel,km,year,gearbox,isofix,led,cruise_control,bluetooth,auto_clim,rear_cam,rear_rad,price
0,CITROEN,C3,PureTech 82ch Shine S&S E6.d,Essence /,17 860 km,/ 2019 /,Manuelle,1,0,1,0,1,0,1,12 799 €
1,RENAULT,Zoe,Zen charge normale R110 Achat Intégral - 20,Electrique /,50 065 km,/ 2020 /,Automatique,1,1,1,1,1,0,0,12 799 €
2,PEUGEOT,3008,HYBRID4 300ch GT Pack e-EAT8,Hybride /,14 456 km,/ 2021 /,Automatique,0,1,1,1,1,1,1,12 799 €
3,PEUGEOT,2008,1.2 PureTech 130ch S&S Style EAT8,Essence /,31 658 km,/ 2022 /,Automatique,1,1,1,1,1,0,1,12 799 €
4,DACIA,Sandero,1.0 ECO-G 100ch Stepway Expression +,Gpl /,13 344 km,/ 2023 /,Manuelle,0,1,1,1,1,1,1,12 799 €


## Nettoyage

In [6]:
df.dtypes

mark              object
model             object
description       object
fuel              object
km                 int32
year               int32
gearbox           object
isofix             int64
led                int64
cruise_control     int64
bluetooth          int64
auto_clim          int64
rear_cam           int64
rear_rad           int64
price              int32
dtype: object

In [7]:
df["km"] = df["km"].str.replace(" km","").str.replace(" ","").astype(int)
df["price"] = df["price"].str.replace(" €","").str.replace(" ","").astype(int)
df["year"] = df["year"].str.replace("/","").astype(int)
df["fuel"] = df["fuel"].str.replace("/","").str.strip()


AttributeError: Can only use .str accessor with string values!

In [ ]:
df['year']=df['year'].astype('int32')
df['km']=df['km'].astype('int32')
df['price']=df['price'].astype('int32')

In [ ]:
print(df.isna().sum())

mark              0
model             0
description       0
fuel              0
km                0
year              0
gearbox           0
isofix            0
led               0
cruise_control    0
bluetooth         0
auto_clim         0
rear_cam          0
rear_rad          0
price             0
dtype: int64


| Dimension      | Contenu                                                               |
| -------------- | --------------------------------------------------------------------- |
| `dim_model`    | mark, model, description                                              |
| `dim_date`     | year, éventuellement décennie ou indicateur « récent »                |
| `dim_gearbox`  | gearbox (Manuelle / Automatique)                                      |
| `dim_features` | isofix, led, cruise_control, bluetooth, auto_clim, rear_cam, rear_rad |


In [8]:
# ---- DIM_MODEL ----
dim_model = df[["mark","model","description"]].drop_duplicates().reset_index(drop=True)
dim_model["model_id"] = dim_model.index + 1

# ---- DIM_DATE ----
dim_date = pd.DataFrame({"year": df["year"].unique()})
dim_date["decade"] = (dim_date["year"] // 10) * 10
dim_date["is_recent"] = (dim_date["year"] > 2018).astype(int)
dim_date["date_id"] = dim_date.index + 1

# ---- DIM_GEARBOX ----
dim_gearbox = pd.DataFrame({"gearbox_type": df["gearbox"].unique()})
dim_gearbox["gearbox_id"] = dim_gearbox.index + 1

# ---- DIM_FEATURES ----
feature_cols = ["isofix","led","cruise_control","bluetooth","auto_clim","rear_cam","rear_rad"]
dim_features = df[feature_cols].drop_duplicates().reset_index(drop=True)
dim_features["features_id"] = dim_features.index + 1


                 ┌─────────────┐
                 │  dim_model  │
                 │─────────────│
                 │ model_id PK │
                 │ mark        │
                 │ model       │
                 │ description │
                 └───────┬─────┘
                         │
                         │
                         │
                 ┌───────▼────────┐
                 │   fact_cars     │
                 │────────────────│
                 │ model_id  FK    │
                 │ date_id   FK    │
                 │ gearbox_id FK   │
                 │ features_id FK  │
                 │ km              │
                 │ price           │
                 └───────┬────────┘
                         │
      ┌──────────────────┼───────────────────┐
      │                  │                   │
      │                  │                   │
┌─────▼─────┐      ┌─────▼──────┐      ┌─────▼────────┐
│  dim_date │      │ dim_gearbox│      │ dim_features │
│───────────│      │────────────│      │─────────────│
│ date_id PK│      │ gearbox_id PK│     │ features_id PK│
│ year      │      │ gearbox_type │     │ isofix        │
│ decade    │      └─────────────┘     │ led           │
│ is_recent │                          │ cruise_control│
└───────────┘                          │ bluetooth     │
                                       │ auto_clim     │
                                       │ rear_cam      │
                                       │ rear_rad      │
                                       └───────────────┘



In [9]:
# Merge pour récupérer les IDs
fact = df.copy()
fact = fact.merge(dim_model, on=["mark","model","description"])
fact = fact.merge(dim_date, on="year")
fact = fact.merge(dim_gearbox, left_on="gearbox", right_on="gearbox_type")
fact = fact.merge(dim_features, on=feature_cols)

# Table de faits finale
fact_table = fact[[
    "model_id",
    "date_id",
    "gearbox_id",
    "features_id",
    "km",
    "price"
]]


In [11]:
dim_model.head()
dim_date.head()
dim_gearbox.head()
dim_features.head()
fact_table.head()


,model_id,date_id,gearbox_id,features_id,km,price
0,1,1,1,1,17860,12799
1,2,2,2,2,50065,12799
2,3,3,2,3,14456,12799
3,4,4,2,4,31658,12799
4,5,5,1,3,13344,12799


In [12]:
dim_date.head()


,year,decade,is_recent,date_id
0,2019,2010,1,1
1,2020,2020,1,2
2,2021,2020,1,3
3,2022,2020,1,4
4,2023,2020,1,5


In [13]:
dim_gearbox.head()


,gearbox_type,gearbox_id
0,Manuelle,1
1,Automatique,2


In [14]:
dim_features.head()


,isofix,led,cruise_control,bluetooth,auto_clim,rear_cam,rear_rad,features_id
0,1,0,1,0,1,0,1,1
1,1,1,1,1,1,0,0,2
2,0,1,1,1,1,1,1,3
3,1,1,1,1,1,0,1,4
4,1,0,1,1,1,1,1,5


In [15]:
fact_table.head()

,model_id,date_id,gearbox_id,features_id,km,price
0,1,1,1,1,17860,12799
1,2,2,2,2,50065,12799
2,3,3,2,3,14456,12799
3,4,4,2,4,31658,12799
4,5,5,1,3,13344,12799
